##### imports

In [57]:
import pandas as pd
import os
import requests
import time
import pprint
from urllib.parse import quote_plus
import re
import json
from Bio.SeqUtils import seq3
import tqdm



### es_iocurie_2014


In [58]:
study = "es_iocurie_2014"


In [59]:
# get data
#TODO import as objects from harvester
## add parameter to skip first 4 lines in patient and study sample data

base_loc = "/Users/costellh/repos/metakb/hmc_notebooks/"
variants = f"{base_loc}/{study}/data_mutations.txt"
meta = f"{base_loc}/{study}/meta_study.txt"
patients = f"{base_loc}/{study}/data_clinical_patient.txt"
samples = f"{base_loc}/{study}/data_clinical_sample.txt"

init_mut_df = pd.read_csv(variants, sep='\t')
init_study_meta = pd.read_csv(meta, sep='\t')
init_patient_df = pd.read_csv(patients, sep='\t', skiprows=4)
init_sample_df = pd.read_csv(samples, sep='\t', skiprows=4)



#### Variant data: subset columns, check for duplicates 

In [60]:
# clean variant data

# subset for necessary columns
mut_df = init_mut_df.filter(['Hugo_Symbol',
                        'Center',
                        'Chromosome',
                        'Start_Position', 
                        'End_Position',
                        'Consequence',
                        'Variant_Classification',
                        'Variant_Type',
                        'Reference_Allele',
                        'Tumor_Seq_Allele2',
                        'Tumor_Sample_Barcode',
                        'Sequence_Source',
                        'HGVSc',
                        'HGVSp',
                        'HGVSp_Short',
                        'Transcript_ID',
                        'RefSeq',
                        'Protein_position'
                       ], axis=1)


# Strip whitespace and retry
mut_df.columns = mut_df.columns.str.strip()
mut_df = mut_df.rename(columns={'Tumor_Sample_Barcode': 'SAMPLE_ID'})

# Check duplicate count
num_duplicates = mut_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# TODO check duplicates for sanity check
# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(mut_df[mut_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# save duplicate rows to file
    dupes = mut_df[mut_df.duplicated(keep=False)]
    mut_df.to_csv('mut_dupes.csv', index=False)
    

    
# remove duplicates, but keep first occurrence
    mut_df = mut_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {mut_df.shape}")
else:
    print("No duplicate rows found.")

    
# print(mut_df.columns)
# # print(mut_df.head())

Number of duplicate rows : 0
No duplicate rows found.


#### Patient data: subset columns, check for duplicates

In [ ]:
# clean patient data
# print(patient_df.shape)
# print(patient_df.columns)

# subset data
# subset for necessary columns
patient_df = init_patient_df.filter(['PATIENT_ID',
                        'AGE',
                        'SEX', 
                        'ETHNICITY',
                        #'Consequence'
                       ], axis=1)

print(patient_df.columns)

# Check duplicate count
num_duplicates = patient_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(patient_df[patient_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# remove duplicates, but keep first occurrence
    patient_df = patient_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {patient_df.shape}")
else:
    print("No duplicate rows found.")

Index(['PATIENT_ID', 'AGE', 'SEX'], dtype='object')
Number of duplicate rows : 0
No duplicate rows found.


#### Sample data: subset columns, check for duplicates

In [ ]:
# clean sample data

# print(sample_df.shape)
# print(sample_df.columns)
# print(sample_df.head())


# subset data
# subset for necessary columns
sample_df = init_sample_df.filter(['PATIENT_ID',
                                     'SAMPLE_ID',
                                     'SAMPLE_CLASS',
                                     'ONCOTREE_CODE',
                                     'CANCER_TYPE',
                                     'CANCER_TYPE_DETAILED',
                                     'TMB_NONSYNONYMOUS'
                                    ], axis=1)


print(sample_df.columns)


# Check duplicate count
num_duplicates = sample_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(sample_df[sample_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# remove duplicates, but keep first occurrence
    sample_df = sample_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {sample_df.shape}")
else:
    print("No duplicate rows found.")

Index(['PATIENT_ID', 'SAMPLE_ID', 'ONCOTREE_CODE', 'CANCER_TYPE',
       'CANCER_TYPE_DETAILED', 'TMB_NONSYNONYMOUS'],
      dtype='object')
Number of duplicate rows : 0
No duplicate rows found.


#### Combine dataframes

In [18]:
# combine dataframes

init_combined_df = mut_df.merge(sample_df, on='SAMPLE_ID', how='left')

# print(mut_df.columns)
# print(mut_df.shape)

# print(sample_df.columns)
# print(sample_df.shape)

# print(init_combined_df.columns)
# print(init_combined_df.shape)

#add patient_df
combined_df = init_combined_df.merge(patient_df, on='PATIENT_ID', how='left')

# print(patient_df.columns)
# print(patient_df.shape)

# print(combined_df.columns)
# print(combined_df.shape)


#### Add STUDY_ID column

In [19]:

# add column for study id
study_id = init_study_meta.iloc[0, 0]
study_id = study_id.replace('cancer_study_identifier: ', '')
# study_id
combined_df['STUDY_ID'] = study_id


#### Check for (and remove) duplicates

In [20]:


# Check duplicate count
num_duplicates = combined_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(combined_df[combined_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# remove duplicates, but keep first occurrence
    combined_df = combined_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {combined_df.shape}")
else:
    print("No duplicate rows found.")



Number of duplicate rows : 0
No duplicate rows found.


#### Remove data from cell lines

In [22]:

# # remove cell lines

# original_shape = combined_df.shape
# print(f"Original shape: {original_shape}")

# #lines to remove
# removed_df = combined_df[combined_df['SAMPLE_CLASS'] == 'Cell line']

# # remove cell lines
# filtered_df = combined_df[combined_df['SAMPLE_CLASS'] != 'Cell line']

# # calculate how many rows were removed
# rows_removed = original_shape[0] - filtered_df.shape[0]
# print(f"Removed {rows_removed} rows where SAMPLE_CLASS == 'Cell line'")

# # print new shape
# print(f"New shape: {filtered_df.shape}")

# # reassign df
# combined_df = filtered_df

# removed_df.to_csv('cell_lines_removed.csv', index=False)
# removed_df.value_counts("SAMPLE_CLASS")


#### Write value counts for NaNs

In [ ]:
# combined_df.isna().sum()

# Hugo_Symbol                   0
# Chromosome                    0
# Start_Position                0
# End_Position                  0
# Consequence                 124 - some of the Variant_Classification=Silent have no consequence described
# Variant_Classification        0
# Variant_Type                  0
# Reference_Allele              0
# Tumor_Seq_Allele2             0
# SAMPLE_ID                     0
# Sequence_Source               0
# HGVSc                       136 - some of the Variant_Classification=Silent and all of the Variant_Classification=3'Flank and 5'Flank
# HGVSp                       346 
# Transcript_ID               124 - some of the Variant_Classification=Silent have no Transcript_ID described
# RefSeq                     1338
# Protein_position            334
# Gnomad_Notation               0
# PATIENT_ID                    0
# SAMPLE_CLASS                  0
# ONCOTREE_CODE                 0
# CANCER_TYPE                   0
# CANCER_TYPE_DETAILED          0
# TMB_NONSYNONYMOUS             0
# AGE                        4745 - some ages undisclosed
# SEX                           0
# ETHNICITY                 10900 - many patients' ethnicities undisclosed
# STUDY_ID                      0


# define output folder (will create it if needed)
output_dir = "value_counts_by_column"
os.makedirs(output_dir, exist_ok=True)

# loop through each column
for col in combined_df.columns:
    # build filename
    filename = f"value_counts_{col}.txt"
    filepath = os.path.join(output_dir, filename)
    
    # write counts to file
    with open(filepath, "w") as f:
        f.write(f"Value counts for column: {col}\n\n")
        f.write(combined_df[col].value_counts(dropna=False).to_string())
        f.write("\n")

print(f"✅ Value counts written for {len(combined_df.columns)} columns to folder: {output_dir}")


✅ Value counts written for 26 columns to folder: value_counts_by_column


In [25]:
combined_df.isna().sum()

Hugo_Symbol                 0
Center                      0
Chromosome                  0
Start_Position              0
End_Position                0
Consequence                 2
Variant_Classification      0
Variant_Type                0
Reference_Allele            0
Tumor_Seq_Allele2           0
SAMPLE_ID                   0
Sequence_Source             0
HGVSc                      78
HGVSp                     885
HGVSp_Short                78
Transcript_ID               2
RefSeq                    350
Protein_position          882
PATIENT_ID                  0
ONCOTREE_CODE               0
CANCER_TYPE                 0
CANCER_TYPE_DETAILED        0
TMB_NONSYNONYMOUS           0
AGE                       157
SEX                       157
STUDY_ID                    0
dtype: int64

#### Replace missing patient, sample data with "No_Data"

In [26]:
#filling in NaNs - AGE, ETHNICITY, Consequence
# TODO add consequence with annotation

# cols_to_fill = ['Consequence', 'AGE', 'ETHNICITY']
# fill_value = "No_Data"

# for col in cols_to_fill:
#     combined_df[col] = combined_df[col].fillna(fill_value)

# combined_df.isna().sum()

combined_df = combined_df.fillna('No_Data')

combined_df.isna().sum()


Hugo_Symbol               0
Center                    0
Chromosome                0
Start_Position            0
End_Position              0
Consequence               0
Variant_Classification    0
Variant_Type              0
Reference_Allele          0
Tumor_Seq_Allele2         0
SAMPLE_ID                 0
Sequence_Source           0
HGVSc                     0
HGVSp                     0
HGVSp_Short               0
Transcript_ID             0
RefSeq                    0
Protein_position          0
PATIENT_ID                0
ONCOTREE_CODE             0
CANCER_TYPE               0
CANCER_TYPE_DETAILED      0
TMB_NONSYNONYMOUS         0
AGE                       0
SEX                       0
STUDY_ID                  0
dtype: int64

#### Construct TEMP GnomAD variant ID column

In [27]:
# construct Gnomad variant ID column
combined_df["temp_Gnomad_Notation"] = combined_df.apply(
    lambda row: f"{row['Chromosome']}-{row['Start_Position']}-{row['Reference_Allele']}-{row['Tumor_Seq_Allele2']}",
    axis=1
)

#### Correcting Chromosome 23 samples to X or Y

##### Write initial combined_df to file

In [28]:
combined_df.to_csv('output0.csv', index=False)

##### Set test variables and REST API variables

In [29]:
# BASE_URL = "https://normalize.cancervariants.org/variation/"
# # HEADERS = {"Accept": "application/json"}
# HEADERS = {
#     "Accept": "application/json",
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"  # mimics a browser
# }
# variant = "23-2408485-G-C"
# gene = "ZBED1"

PATTERN = re.compile(r'^23-')        # anchored ^ so only the chromosome prefix is substituted



##### FUNCTION: Flag rows with Chrom23

In [30]:
def flag_rows_chrom_23(df):
    """
    Create "Chrom_23" column, True for those with Chromosome = 23

    Parameters
    ----------
    df : pd.DataFrame
        Must contain column 'Chromosome'.
    
    Returns
    -------
    dataframe
    """
    df["Chrom_23"] = False
    # print(combined_df.head)
    df["Chrom_23"] = df["Chromosome"].astype(str).str.strip().eq("23")
    df.loc[df["Chromosome"] == 23, "Chrom_23"] = True
    # print(combined_df["Chrom_23"].value_counts())
    # print(combined_df["Chromosome"].value_counts())
    
    return df

combined_df = flag_rows_chrom_23(combined_df)

combined_df.to_csv("output_flag_chrom_23.csv", index=False)

##### FUNCTION: change female chrom23 to X

In [31]:
# print(combined_df["Chromosome"].value_counts())

def chr23_female(df):
    """
    Convert Chromosome 23 to 'X' for rows where SEX is female.
    
    Parameters
    ----------
    df : pd.DataFrame
        Must contain columns 'Chromosome' and 'SEX'.
    
    Returns
    -------
    dataframe
    """
    # Ensure we’re comparing like with like
    chr_col = df["Chromosome"].astype(str).str.strip()
    sex_col = df["SEX"].astype(str).str.upper().str.strip()   # handles 'F', 'f', 'Female', etc.
    
    mask = (chr_col == "23") & (sex_col.str.startswith("F"))
    df.loc[mask, "Chromosome"] = "X"
    return df

combined_df = chr23_female(combined_df)

# print(combined_df["Chromosome"].value_counts())

combined_df.to_csv('output1_femaleX.csv', index=False)

##### FUNCTION: Add cols for Chr23_X and Chr23_Y, fill with false

In [32]:
def add_cols_chrom_23_male(df):
    """
    Create "Chr23_X and Chr23_Y" columns, fill with false

    Parameters
    ----------
    df : pd.DataFrame
        Must contain column 'Chromosome'.
    
    Returns
    -------
    dataframe
    """
    df["Chr23_X"] = False
    df["Chr23_Y"] = False
    # df["Chrom_23"] = df["Chromosome"].astype(str).str.strip().eq("23")
    # df.loc[df["Chromosome"] == 23, "Chrom_23"] = True
    # print(combined_df["Chr23_X"].value_counts())
    # print(combined_df["Chr23_Y"].value_counts())
    return df

combined_df = add_cols_chrom_23_male(combined_df)
combined_df.to_csv('output2_new_chr23_boolean_cols.csv', index=False)

##### FUNCTION: Adjust GnomAD variant to accept X

In [33]:
def chr23_to_X(variant: str) -> str:
    """Convert a single '23-' prefix in a variant string to 'X-'."""
    return PATTERN.sub('X-', variant) if isinstance(variant, str) else variant

# x_variant = chr23_to_X(variant)
# print(x_variant)

##### FUNCTION: Adjust GnomAD variant to accept Y

In [34]:
def chr23_to_Y(variant: str) -> str:
    """Convert a '23-' prefix in a single variant string to 'Y-'."""
    return PATTERN.sub('Y-', variant) if isinstance(variant, str) else variant

# y_variant = chr23_to_Y(variant)
# print(y_variant)

##### FUNCTION: Test tokenization

In [35]:
def test_variant_tokenization(variant: str, delay=0.5):
    """
    Fetch normalized variant info from VICC API for a single variant string.

    Parameters
    ----------
    variant : str
        A GnomAD-style variant (e.g., '23-2408485-G-C').
    delay : float
        Seconds to wait between API requests (default 0.5).

    Returns
    -------
    pd.DataFrame
        DataFrame with original variant and raw JSON string response.
    """
    
    # BASE_URL = "https://normalize.cancervariants.org/variation/"
    BASE_URL = "http://localhost:8001/variation/"
    HEADERS = {
        "Accept": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"  # mimics a browser
        }
    
    results = []

    # url = f"{BASE_URL}normalize?q={variant}"
    url = f"{BASE_URL}normalize?q={variant}&hgvs_dup_del_mode=default&input_assembly=GRCh37"

    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            results.append({
                "variant": variant,
                "response": json.dumps(data)
            })
        else:
            results.append({
                "variant": variant,
                "response": f"Error {response.status_code}: {response.text}"
            })
    except Exception as e:
        results.append({
            "variant": variant,
            "response": f"Exception: {str(e)}"
        })

    time.sleep(delay)

    return pd.DataFrame(results)


##### FUNCTION: Check if variant on X

In [36]:
def check_for_x_variant(df, variant):
    # Convert the variant to X-style format (e.g., "23-..." → "X-...")
    variant_x = chr23_to_X(variant)

    # Query the API and get a one-row DataFrame
    x_df = test_variant_tokenization(variant_x)  # returns a one-row DataFrame

    # Extract and parse JSON string from the response
    raw_response = x_df.loc[0, "response"]

    try:
        parsed_response = json.loads(raw_response)
    except json.JSONDecodeError:
        print(f"❌ Failed to parse JSON for: {variant_x}")
        return df

    # Ensure 'variation' is in the parsed response
    if "variation" not in parsed_response:
        print(f"⚠️ 'variation' key not found for: {variant_x}")
        return df

    # Try to extract hgnc_id if available
    hgnc_id = None
    try:
        hgnc_id = parsed_response["variation"]["extensions"][0]["value"][0]["hgnc_id"]
        print(f"✅ Extracted HGNC ID for {variant_x}: {hgnc_id}")
    except (KeyError, IndexError, TypeError):
        print(f"⚠️ No HGNC ID found for: {variant_x}")

    # Initialize columns if needed
    if "x_hgnc_id" not in df.columns:
        df["x_hgnc_id"] = "no_value"
    if "Chr23_X" not in df.columns:
        df["Chr23_X"] = False

    # Reconstruct variant string from each row to match normalized variant_x
    reconstructed = (
        "X-" +
        df["Start_Position"].astype(str).str.strip() + "-" +
        df["Reference_Allele"].astype(str).str.strip() + "-" +
        df["Tumor_Seq_Allele2"].astype(str).str.strip()
    )

    # Mask: match rows on reconstructed X-variant and Chromosome == 23
    chrom_col = df["Chromosome"].astype(str).str.strip()
    mask = (chrom_col == "23") & (reconstructed == variant_x)

    print(f"🧪 Matched {mask.sum()} row(s) for {variant_x}")

    # Set Chr23_X = True
    df.loc[mask, "Chr23_X"] = True

    # Set x_hgnc_id if available
    if hgnc_id is not None:
        df.loc[mask, "x_hgnc_id"] = hgnc_id

    return df






##### FUNCTION: Check if variant on Y

In [37]:
def check_for_y_variant(df, variant):
    # Convert the variant to Y-style format (e.g., "23-..." → "Y-...")
    variant_y = chr23_to_Y(variant)

    # Query the API and get a one-row DataFrame
    y_df = test_variant_tokenization(variant_y)  # returns a one-row DataFrame

    # Extract and parse JSON string from the response
    raw_response = y_df.loc[0, "response"]

    try:
        parsed_response = json.loads(raw_response)
    except json.JSONDecodeError:
        print(f"❌ Failed to parse JSON for: {variant_y}")
        return df

    # Ensure 'variation' is in the parsed response
    if "variation" not in parsed_response:
        print(f"⚠️ 'variation' key not found for: {variant_y}")
        return df

    # Try to extract hgnc_id if available
    hgnc_id = None
    try:
        hgnc_id = parsed_response["variation"]["extensions"][0]["value"][0]["hgnc_id"]
        print(f"✅ Extracted HGNC ID for {variant_y}: {hgnc_id}")
    except (KeyError, IndexError, TypeError):
        print(f"⚠️ No HGNC ID found for: {variant_y}")

    # Initialize columns if needed
    if "y_hgnc_id" not in df.columns:
        df["y_hgnc_id"] = "no_value"
    if "Chr23_Y" not in df.columns:
        df["Chr23_Y"] = False

    # Reconstruct variant string from each row to match normalized variant_y
    reconstructed = (
        "Y-" +
        df["Start_Position"].astype(str).str.strip() + "-" +
        df["Reference_Allele"].astype(str).str.strip() + "-" +
        df["Tumor_Seq_Allele2"].astype(str).str.strip()
    )

    # Mask: match rows on reconstructed Y-variant and Chromosome == 23
    chrom_col = df["Chromosome"].astype(str).str.strip()
    mask = (chrom_col == "23") & (reconstructed == variant_y)

    print(f"🧪 Matched {mask.sum()} row(s) for {variant_y}")

    # Set Chr23_Y = True
    df.loc[mask, "Chr23_Y"] = True

    # Set y_hgnc_id if available
    if hgnc_id is not None:
        df.loc[mask, "y_hgnc_id"] = hgnc_id

    return df



##### FUNCTION: Master function for dealing with male chrom23 

In [38]:
def chr23_male(df, variant):
    # df = add_cols_chrom_23_male(df)          # prep chromosome-23 columns
    if "Chr23_X" not in df.columns:
        df["Chr23_X"] = False
    if "Chr23_Y" not in df.columns:
        df["Chr23_Y"] = False
    df = check_for_x_variant(df, variant)    # pass **both** args
    df = check_for_y_variant(df, variant)

    df.to_csv("output2_new_chr23_boolean_cols.csv", index=False)
    return df


##### Setting chromosome 23 variant notations as a list


In [39]:

chrom_23_list = combined_df[
    (combined_df["SEX"].str.lower() == "male") &
    (combined_df["Chrom_23"] == True)
]["temp_Gnomad_Notation"].dropna().tolist()

print("Total variants in list:", len(chrom_23_list))
print("Sample of chrom_23_list:", chrom_23_list[:5])

# print(len(chrom_23_list))

# with open("chrom_23_list.json", "w") as f:
#     json.dump(chrom_23_list, f, indent=2)

Total variants in list: 0
Sample of chrom_23_list: []


##### Initialize x_hgnc_id and y_hgnc_id columns


In [40]:
if "x_hgnc_id" not in combined_df.columns:
    combined_df["x_hgnc_id"] = "no_value"
if "y_hgnc_id" not in combined_df.columns:
    combined_df["y_hgnc_id"] = "no_value"

##### RUN: driver function

In [41]:
# Start with a copy of your combined_df
result_df = combined_df.copy()

# Iterate through each chromosome 23 variant for male samples
for variant in chrom_23_list:
    print(f"▶️ Checking variant: {variant}")  # Add this line
    result_df = chr23_male(result_df, variant)

# Now all updates are preserved in result_df
print(result_df["Chr23_X"].value_counts(dropna=False))
print(result_df["Chr23_Y"].value_counts(dropna=False))

# Save the final output to CSV
result_df.to_csv("output_post_23_BOOLEAN.csv", index=False)

##TODO: THIS IS ONLY CHANGING ZNF449 TO TRUE FOR X
##TODO: SOME GENES SHOW ALIASES, SOME ARE WRONG GENES
##TODO: MUST TRUST THAT NORMALIZER IS DOING ITS JOB. WHEN A NEW SYMBOL COMES BACK, BUT THEY ARE ALIASES, NORMALIZER GIVES 
##TODO: USE GENEID INSTEAD OF GENE SYMBOL FOR TOKENIZATION TEST. DO VARIATION TESTER FOR GENE SYMBOL AND THEN GET GENE ID. THEN TEST VARIANT, GET GENE ID. IF THEY ARE THE SAME, RECORD.  
##TODO: why aren't some of the x variants showing up with hgnc IDs.  Like X-84258927-C-T

Chr23_X
False    2043
Name: count, dtype: int64
Chr23_Y
False    2043
Name: count, dtype: int64


##### FUNCTION: Reassign male chrom 23s

In [42]:
def correct_male_chrom23(df):
    # Initialize ambig_chrom column
    df["ambig_chrom"] = "non-ambiguous"

    def update_row(row):
        if row.get("Chrom_23") is True and str(row.get("SEX", "")).strip().lower() == "male":
            if row["Chr23_X"] and not row["Chr23_Y"]:
                row["Chromosome"] = "X"
            elif row["Chr23_Y"] and not row["Chr23_X"]:
                row["Chromosome"] = "Y"
            elif row["Chr23_X"] and row["Chr23_Y"]:
                row["ambig_chrom"] = "XY"
            else:  # neither Chr23_X nor Chr23_Y is True
                row["ambig_chrom"] = "neither"
        return row

    # Apply corrections row-by-row
    df = df.apply(update_row, axis=1)

    # Check for ambiguous rows
    ambig_rows = df[df["ambig_chrom"].isin(["XY", "neither"])]
    if not ambig_rows.empty:
        print(f"⚠️ Found {len(ambig_rows)} row(s) with ambiguous chromosome identification.")
        print(ambig_rows[["temp_Gnomad_Notation", "Hugo_Symbol", "ambig_chrom"]].head())

    return df




##### RUN: Correct male 23s and see if any ambiguous chromosome 23 values exist

In [43]:
result_df = correct_male_chrom23(result_df)

print(result_df["ambig_chrom"].value_counts())

result_df.to_csv("output_post_23_correction.csv", index=False)

# result_df 


ambig_chrom
non-ambiguous    2043
Name: count, dtype: int64


##### FUNCTION: correct ambiguous chromosomes

In [44]:
def resolve_ambiguous_chromosomes(df):
    """
    Resolve ambiguous Chr23 variants in male samples and flag problematic rows.

    - For Chrom_23 == True and SEX == male:
      - If both x_hgnc_id and y_hgnc_id have values → ⚠️ warn
      - If neither has values → ❌ warn
      - If ambig_chrom == "XY" or "neither", try to resolve Chromosome field
    """
    chrom23_male_mask = (
        (df["Chrom_23"] == True) &
        (df["SEX"].str.lower().str.strip() == "male")
    )

    for idx, row in df[chrom23_male_mask].iterrows():
        x_id = str(row.get("x_hgnc_id", "")).strip()
        y_id = str(row.get("y_hgnc_id", "")).strip()
        ambig_status = row.get("ambig_chrom", "not_set")

        # ⚠️ Raise general warnings, regardless of ambig_chrom
        if x_id != "no_value" and y_id != "no_value":
            print(f"⚠️ Row {idx}: Both x_hgnc_id and y_hgnc_id have values. Manual check recommended.")
        elif x_id == "no_value" and y_id == "no_value":
            print(f"❌ Row {idx}: Neither x_hgnc_id nor y_hgnc_id has a value.")

        # ✅ Try resolving ambiguous chromosomes
        if ambig_status in ["XY", "neither"]:
            if x_id != "no_value" and y_id == "no_value":
                df.at[idx, "Chromosome"] = "X"
                df.at[idx, "ambig_chrom"] = "resolved_X"
                print(f"✅ Resolved index {idx} to Chromosome X")
            elif y_id != "no_value" and x_id == "no_value":
                df.at[idx, "Chromosome"] = "Y"
                df.at[idx, "ambig_chrom"] = "resolved_Y"
                print(f"✅ Resolved index {idx} to Chromosome Y")
            elif x_id != "no_value" and y_id != "no_value":
                print(f"⚠️ Index {idx}: Ambiguous Chromosome with both HGNC IDs present.")
            else:
                print(f"❌ Index {idx}: Ambiguous Chromosome with no HGNC ID found.")

    return df




##### RUN: resolve ambiguous chromosomes

In [45]:
resolve_ambiguous_chromosomes(result_df)

,Hugo_Symbol,Center,Chromosome,Start_Position,End_Position,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele2,...,AGE,SEX,STUDY_ID,temp_Gnomad_Notation,Chrom_23,Chr23_X,Chr23_Y,x_hgnc_id,y_hgnc_id,ambig_chrom
0,KANK4,Institut Curie,1,62728781,62728781,missense_variant,Missense_Mutation,SNP,T,A,...,13.7,Male,es_iocurie_2014,1-62728781-T-A,False,False,False,no_value,no_value,non-ambiguous
1,OR51G2,Institut Curie,11,4936278,4936278,missense_variant,Missense_Mutation,SNP,A,T,...,13.7,Male,es_iocurie_2014,11-4936278-A-T,False,False,False,no_value,no_value,non-ambiguous
2,ANO3,Institut Curie,11,26682872,26682872,3_prime_UTR_variant,3'UTR,SNP,C,T,...,13.7,Male,es_iocurie_2014,11-26682872-C-T,False,False,False,no_value,no_value,non-ambiguous
3,MRP63,Institut Curie,13,21751981,21751981,3_prime_UTR_variant,3'UTR,SNP,G,A,...,13.7,Male,es_iocurie_2014,13-21751981-G-A,False,False,False,no_value,no_value,non-ambiguous
4,C14orf73,Institut Curie,14,103568642,103568642,synonymous_variant,Silent,SNP,C,A,...,13.7,Male,es_iocurie_2014,14-103568642-C-A,False,False,False,no_value,no_value,non-ambiguous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2038,RFX7,Institut Curie,15,56386925,56386929,frameshift_variant,Frame_Shift_Del,DEL,GTGTA,-,...,7.5,Female,es_iocurie_2014,15-56386925-GTGTA--,False,False,False,no_value,no_value,non-ambiguous
2039,SNTB1,Institut Curie,8,121824083,121824084,5_prime_UTR_variant,5'UTR,INS,-,CCGCCA,...,7.5,Female,es_iocurie_2014,8-121824083---CCGCCA,False,False,False,no_value,no_value,non-ambiguous
2040,GTF2I,Institut Curie,7,74105345,74105349,frameshift_variant,Frame_Shift_Del,DEL,TTGCA,-,...,19.5,Male,es_iocurie_2014,7-74105345-TTGCA--,False,False,False,no_value,no_value,non-ambiguous
2041,NUP62CL,Institut Curie,X,106397473,106397477,"splice_acceptor_variant,coding_sequence_variant",Splice_Site,DEL,TACAC,-,...,19.5,Male,es_iocurie_2014,X-106397473-TACAC--,False,False,False,no_value,no_value,non-ambiguous


##### FUNCTION: test gene tokenization


In [46]:
def test_gene_tokenization(gene: str, delay=0.5):
    BASE_URL = "https://normalize.cancervariants.org/gene/"
    HEADERS = {
        "Accept": "application/json",
        "User-Agent": "Mozilla/5.0"
    }

    results = []

    url = f"{BASE_URL}normalize?q={gene}"

    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            results.append({
                "gene": gene,
                "response": json.dumps(data)
            })
        else:
            results.append({
                "gene": gene,
                "response": f"Error {response.status_code}: {response.text}"
            })
    except Exception as e:
        results.append({
            "gene": gene,
            "response": f"Exception: {str(e)}"
        })

    time.sleep(delay)
    return pd.DataFrame(results)


##### FUNCTION: test gene tokenization

In [47]:

def update_gene_hgnc_id_inplace(df):
    for idx, row in df.iterrows():
        if row.get("Chrom_23") and str(row.get("SEX", "")).strip().lower() == "male":
            try:
                result = test_gene_tokenization(row["Hugo_Symbol"])
                full_id = result["gene"]["id"]  # e.g., "normalize.gene.hgnc:447"
                hgnc_id = full_id.split(":")[-1]  # Extract "447"
                df.at[idx, "gene_hgnc_id"] = hgnc_id
            except Exception as e:
                print(f"⚠️ Could not extract HGNC ID for {row['Hugo_Symbol']}: {e}")

    print(df.head())
    return df


##### Setting genes as list


In [49]:

# Ensure column exists and is initialized to "untested"
if "gene_hgnc_id" not in result_df.columns:
    result_df["gene_hgnc_id"] = "untested"

gene_list = result_df[
    (result_df["SEX"].str.lower() == "male") &
    (result_df["Chrom_23"] == True)
]["Hugo_Symbol"].dropna().tolist()

print("Total genes in list:", len(gene_list))
print("Sample of gene_list:", gene_list[:5])

Total genes in list: 0
Sample of gene_list: []


##### RUN: Gene tokenization

In [50]:
# Start with a copy of your combined_df
result_post_23_df = result_df.copy()

def populate_gene_hgnc_col(gene_list, df):
    # Add the column once
    if "gene_hgnc_id" not in df.columns:
        df["gene_hgnc_id"] = None

    for gene in gene_list:
        print(f"▶️ Checking gene: {gene}")
        gene_df = test_gene_tokenization(gene)

        # Extract and parse JSON string from the response
        raw_response = gene_df.loc[0, "response"]
        try:
            parsed_response = json.loads(raw_response)
        except json.JSONDecodeError:
            print(f"❌ Failed to parse JSON for: {gene}")
            continue

        if "gene" not in parsed_response:
            print(f"⚠️ 'gene' key not found for: {gene}")
            continue

        try:
            hgnc_id = parsed_response["gene"]["id"].split(":")[-1]
            print(f"✅ Extracted HGNC ID for {gene}: {hgnc_id}")
        except (KeyError, IndexError, TypeError):
            print(f"⚠️ No HGNC ID found for: {gene}")
            continue

        # Apply to matching rows
        df.loc[
            (df["Chrom_23"] == True) & 
            (df["SEX"].str.strip().str.lower() == "male") & 
            (df["Hugo_Symbol"].str.strip() == gene.strip()),
            "gene_hgnc_id"
        ] = hgnc_id

    return df




result_post_23_df = populate_gene_hgnc_col(gene_list, result_post_23_df)
print(result_post_23_df["gene_hgnc_id"].value_counts(dropna=False))
result_post_23_df.to_csv("output_post_gene_norm.csv", index=False)



gene_hgnc_id
untested    2043
Name: count, dtype: int64


##### FUNCTION: check hgnc_id matches

In [51]:
def validate_gene_hgnc_match(df):
    """
    Compare gene_hgnc_id to x_hgnc_id or y_hgnc_id for Chromosome 23 variants in male samples.
    
    Adds a column 'hgnc_id_match':
        - "untested" (default)
        - set to matching hgnc_id if match is found
        - "no_match" if mismatch is found

    Prints warnings for mismatches.
    """
    # Initialize column
    if "hgnc_id_match" not in df.columns:
        df["hgnc_id_match"] = "untested"

    # Define mask for male Chr23 variants
    mask = (df["Chrom_23"] == True) & (df["SEX"].str.strip().str.lower() == "male")

    for idx, row in df[mask].iterrows():
        gene_id = str(row.get("gene_hgnc_id", "")).strip()
        chrom = str(row.get("Chromosome", "")).strip()
        x_id = str(row.get("x_hgnc_id", "")).strip()
        y_id = str(row.get("y_hgnc_id", "")).strip()

        if chrom == "X":
            if gene_id == x_id and gene_id != "no_value":
                df.at[idx, "hgnc_id_match"] = gene_id
            else:
                df.at[idx, "hgnc_id_match"] = "no_match"
                print(f"❌ Row {idx}: X chromosome mismatch. gene_hgnc_id={gene_id}, x_hgnc_id={x_id}")
        elif chrom == "Y":
            if gene_id == y_id and gene_id != "no_value":
                df.at[idx, "hgnc_id_match"] = gene_id
            else:
                df.at[idx, "hgnc_id_match"] = "no_match"
                print(f"❌ Row {idx}: Y chromosome mismatch. gene_hgnc_id={gene_id}, y_hgnc_id={y_id}")

    return df


##### RUN: validate hgnc ID matches

In [52]:
post_validation_df = validate_gene_hgnc_match(result_post_23_df)
post_validation_df.to_csv("output_post_validation.csv", index=False)

##### Create Gnomad_Notation column and populate.

In [53]:
# construct Gnomad variant ID column
post_validation_df["Gnomad_Notation"] = post_validation_df.apply(
    lambda row: f"{row['Chromosome']}-{row['Start_Position']}-{row['Reference_Allele']}-{row['Tumor_Seq_Allele2']}",
    axis=1
)

#### Function: fill in missing values in df

##### FUNCTION: Query API for HGVSp and RefSeq

In [54]:
#check NAs 
# post_validation_df.isna().sum()
variant = "4-159603535-G-C"
def query_gnomad_vcf_to_protein(variant: str, delay=0.5):
    BASE_URL = "http://localhost:8001/variation/"
    HEADERS = {
        "Accept": "application/json",
        "User-Agent": "Mozilla/5.0"
    }

    results = []

    url = f"{BASE_URL}gnomad_vcf_to_protein?q={variant}"

    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            results.append({
                "variant": variant,
                "response": json.dumps(data)
            })
        else:
            results.append({
                "variant": variant,
                "response": f"Error {response.status_code}: {response.text}"
            })
    except Exception as e:
        results.append({
            "variant": variant,
            "response": f"Exception: {str(e)}"
        })

    time.sleep(delay)
    return pd.DataFrame(results)

r = query_gnomad_vcf_to_protein(variant)
parsed_response = json.loads(r.loc[0, "response"])
print(parsed_response)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

##### FUNCTION: update missing RefSeq values in df

In [55]:
# def get_protein_position(parsed_response):
#     protein_position = r["variation"]["location"]["end"]
#     print(protein_position)
#     return protein_position

In [130]:
# def get_ref_aa(parsed_response):
#     ref_aa = r['variation']['location']['sequence']
#     print(ref_aa)
#     return ref_aa

In [131]:
# def get_var_aa(parsed_response):
#     var_aa = r['variation']['state']['sequence']
#     print(var_aa)
#     return var_aa

In [ ]:
# # Ensure column exists and is initialized to "untested"
# if "temp_RefSeq" not in post_validation_df.columns:
#     post_validation_df["temp_RefSeq"] = "untested"

# # print(post_validation_df.columns)
# # post_validation_df['temp_RefSeq'].head(n=5) 

# # put "testing" in temp col for rows with no RefSeq value
# post_validation_df.loc[pd.isna(post_validation_df["RefSeq"]), "temp_RefSeq"] = "testing"
# post_validation_df["temp_RefSeq"].value_counts(dropna=False)

# #get list of variants to test
# refseq_testing_variants = post_validation_df.loc[
#     post_validation_df["temp_RefSeq"] == "testing", "Gnomad_Notation"
# ].tolist()



In [133]:
# # Driver function
# def driver_fun_RefSeq(variant, df):
#     try:
#         # Call your API
#         response_df = query_gnomad_vcf_to_protein(variant)
#         response_data = json.loads(response_df.loc[0, "response"])

#         # Try to extract the RefSeq from response
#         refseq = next(
#             (m["coding"]["code"] for m in response_data.get("gene_context", {}).get("mappings", [])
#              if m.get("coding", {}).get("id", "").startswith("refseq:")),
#             None
#         )

#         # Update the temp_RefSeq column in-place
#         df.loc[
#             df["Gnomad_Notation"] == variant, "temp_RefSeq"
#         ] = refseq if refseq else "not_found"

#         time.sleep(0.25)  # be polite to API

#     except Exception as e:
#         print(f"Error processing {variant}: {e}")
#         df.loc[
#             df["Gnomad_Notation"] == variant, "temp_RefSeq"
#         ] = "error"
#     return df


# for variant in refseq_testing_variants:
#     refseq_df = driver_fun_RefSeq(variant, post_validation_df)

# refseq_df.to_csv("output_post_TEMP_refseq_entries.csv", index=False)

##### Next step?

In [134]:


#get missing p dots!

#Need function that tkes gnomad notation and puts it through normalizer, then takes response and fashions it into the test fixture
#put response into columns "focus variant members", etc. 

#figure out other info for object - cohort, frequency, etc. 








In [56]:
# remove variant dupes per patient

# find duplicated (PATIENT_ID, Gnomad_Notation) pairs
dupe_mask = post_validation_df.duplicated(subset=["PATIENT_ID", "Gnomad_Notation"], keep="first")
# new DataFrame with the duplicated rows
patient_variant_dupes = post_validation_df[dupe_mask]
# remove those rows from the original DataFrame
post_validation_df_cleaned = post_validation_df[~dupe_mask]
# write removed rows to file
patient_variant_dupes.to_csv("patient_variant_dupes.csv", index=False)
# print the number of rows removed
print(f"Removed {patient_variant_dupes.shape[0]} rows with duplicated Gnomad_Notation per PATIENT_ID.")
# reassign dataframe:
# combined_df = combined_df_cleaned
#write dataframe to file
post_validation_df_cleaned.to_csv("post_validation_df_cleaned.csv", index=False)

Removed 0 rows with duplicated Gnomad_Notation per PATIENT_ID.
